# TOP

In [1]:
# full screen width of Jupyter notebook
from IPython.display import display, HTML
display(HTML("<style>.container {width:80%;} </style>")) 
display(HTML("<style>.toc{ margin-top: 100px;} </style>"))

In [2]:
import os
os.environ['rz_token'] = '5bc2502a20241231'
os.chdir('../../')

In [3]:
import os
print(f'[+] cwd = {os.getcwd()}, __file__ = {__file__ if "__file__" in vars() else None}')

import sys
sys.path.insert(0, os.path.abspath('./'))
import lightgbm as lgb
import json
import os
import runningz as rz
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from runningz.fe.util.util import subsample
from runningz.fe.prepare_data.basic_data import *
from runningz.fe.er import Typer
from runningz.util.clss import get_subclass_by_name
from runningz.util.const import CONST
from runningz.util.util import my_print as print
from runningz.util.util import dict_2_str
from runningz.core.op import show_op_dict
from runningz.fe.config.fm_config import fm_config_list_ALL, data_config
from runningz.fe.util.util import print_history
rz.seed_everything(42)
OP_DICT, OP_DICT_NOT, OP_DICT_YES = show_op_dict()
CONST.DEBUG = False
CONST.MY_PRINT = False

[+] cwd = /Users/runningz/project/runningz, __file__ = None
[+] len(fm_config_list_TOP) = 18
[+] len(fm_config_list_OPT) = 100
[RZ][22:51:25] runningz/core/op/__init__.py:218-> [+] OP_DICT      = 377
[RZ][22:51:25] runningz/core/op/__init__.py:219-> [+] OP_DICT_NOT  = 100
[RZ][22:51:25] runningz/core/op/__init__.py:220-> [+] OP_DICT_YES  = 277
[RZ][22:51:25] runningz/core/op/__init__.py:220-> x             0
[RZ][22:51:25] runningz/core/op/__init__.py:220-> roll         10
[RZ][22:51:25] runningz/core/op/__init__.py:220-> agg          24
[RZ][22:51:25] runningz/core/op/__init__.py:220-> gp           19
[RZ][22:51:25] runningz/core/op/__init__.py:220-> roll_2       12
[RZ][22:51:25] runningz/core/op/__init__.py:220-> agg_2         9
[RZ][22:51:25] runningz/core/op/__init__.py:220-> gp_2          9
[RZ][22:51:25] runningz/core/op/__init__.py:220-> merge         5
[RZ][22:51:25] runningz/core/op/__init__.py:220-> op_1         13
[RZ][22:51:25] runningz/core/op/__init__.py:220-> op_1_v    

In [4]:
# sys.argv = 'fe_main.py --task DEBUG --label twap 0 1'.split(' ')
sys.argv = 'res_main.py --task DEBUG --label twap 0 1'.split(' ')

In [5]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--task', type=str, default='DEBUG')  # str
parser.add_argument('--label', nargs=3, default=['vwap', 0, 1], help='help info') # any number of
parser.add_argument('--debug', nargs='*', default=None, help='help info') # any number of
args = parser.parse_args()
print(f'[+] args = {args}')

CONST.DEBUG = (type(args.debug) is list)
task = args.task
data_config.other_info.col_label_args  = args.label
data_config.other_info.col_label_shift = f'{args.label[0]}_ret'

# path_output
date = rz.time_str()[:10].replace('/', '')
path_output = f'runningz/fe/_cache_/{date}/'
os.makedirs(path_output, exist_ok = True)
# import IPython; IPython.embed()

# ------------------------------ read ------------------------------
# read data
print(dict_2_str(data_config))
dm = get_subclass_by_name(BaseData, data_config.data_reader)().load(data_config)
print('[init] <---------- mask_ALL  ----------> ')
dm.set_mask_ALL(verbose = True) # for multi predict
print(f'[+] dm.shape() = {dm.shape()}')
if CONST.DEBUG: 
    print('[init] <---------- subsample  ---------> ')
    subsample(dm, pct = 0.1)
    print(f'[+] dm.shape() = {dm.shape()}')

# ------------------------------ cols_dict_init ------------------------------
col_id = data_config.cols_dict.col_id
col_date = data_config.other_info.col_date
col_label = data_config.cols_dict.col_label
cols_tid = data_config.cols_dict.cols_tid
col_label_shift = data_config.other_info.col_label_shift
col_tid = data_config.cols_dict.col_tid

cols_dict_init = {}
history = []
path = None
if path is not None:
    cols_dict_init = rz.load_json(path)
    history = rz.load_json(f'{path[:-5]}.history')
    print(f'[+] cache, dict = ')
    print(pd.Series(cols_dict_init).apply(len).to_string())
    print(f'[+] cache, history = ')
    print_history(history)

# ---------- basic ----------
print('[init] <---------- fe_basic  ----------> ')
dm.fe_go(cols_dict_init.get('name_list_basic', []))
dm.fe_go(cols_dict_init.get('cols_fea', []))
name_list_basic = []
name_list = [
    f'y_shift_1 = roll(shift, {col_id}, {col_label_shift}, {col_tid}, 1)',
    f'y_shift_2 = roll(shift, {col_id}, {col_label_shift}, {col_tid}, 2)',
    f'y_shift_3 = roll(shift, {col_id}, {col_label_shift}, {col_tid}, 3)',
]
dm.fe_go(name_list)
name_list_basic += name_list
cols_fix = []
cols_dict_init['cols_fix'] = cols_fix

# ------------------------------ cache ------------------------------
typer_config = {
    **dict(data_config.cols_dict),
    'cols_num': [],
    'cols_cat': [col_id, col_tid] +  (cols_tid if len(cols_tid) != 1 else []),
    'cols_datetime': [],
    'cols_timestamp': [],
    'cols_fix': cols_fix,
    'name_list_basic': name_list_basic,
    'cols_fea': [],
}
typer_config['data_config'] = dict(data_config)
typer_config = {**typer_config, **cols_dict_init}  # insert data_config into cols_dict
typer = Typer(typer_config)
typer.update_cols_num(dm.head().columns)
print('[init] <---------- typer_config ----------> ')
print(pd.Series(typer.typer_config).apply(len).to_string())
# import IPython; IPython.embed()

# # ------------------------------ go ------------------------------
fm_config_list = fm_config_list_ALL
fm = rz.fe.FEMaster(dm, typer, history)
# for i, fm_config in tqdm(enumerate(fm_config_list)):
#     # if i <= 76: continue

#     time_str = rz.time_str(fmt = '%Y%m%d_%H%M%S')
#     print(f'{">>" * 50}, fm_config_i = {i:02d} / {len(fm_config_list):02d} | time_str = {time_str}')
#     fm.run(fm_config_list[i: i + 1])

#     if len(fm.history) > 0:
#         cols_fea = fm.typer.get_cols_all('fea')
#         score = pd.DataFrame(fm.history)['best'].iloc[-1]['a_score'][-1]
#         path = f'{path_output}/{task}_{time_str}_{score:.4f}_{len(cols_fea)}.dict'
#         fm.typer.dump_config(path)
#         rz.dump_json(fm.history, f'{path[:-5]}.history')

# # rz.dump_pickle(fm, f'{path_output}/fm.bin')
print('[+] done!')

[+] args = Namespace(task='DEBUG', label=['twap', '0', '1'], debug=None)
{
    "data_reader": "BasicData",
    "path_input": "/Users/runningz/workspace/data/tushare/daily",
    "date_dict": {
        "all": [20190101, 20231231],
        "hist": [20100101, 20191231],
        "train": [20100101, 20191231],
        "test": [20200101, 20221231],
        "valid": [20200101, 20221231],
        "look": [20200101, 20221231],
        "eval": [20230101, 20231231]
    },
    "cols_dict": {
        "col_id": "ts_code",
        "col_tid": "trade_date",
        "col_label": "label",
        "cols_tid": ["trade_date"],
        "cols_not": ["mask"],
        "cols_label": [],
        "cols_mask": []
    },
    "other_info": {
        "col_date": "trade_date",
        "cols_ttt_list": [],
        "col_label_args": ["twap", "0", "1"],
        "col_label_shift": "twap_ret",
        "col_label_from": null,
        "f_label_pre": "cszscore",
        "shift_i": 1,
        "cols_shift_1_not": []
    },
    "n

read_csv_list(): 100%|################################################| 1/1 [00:10<00:00, 10.75s/it]


[init] <---------- label ----------> 
[+] cols_label = ['LABEL_twap_0_1', 'LABEL_RAW']
[+] f_label_pre = cszscore, col_label_from = LABEL_twap_0_1
[+] total = 1


fe_generator:   0%|          | 0/1 [00:00<?, ?it/s]

[+] total = 1


...

[+] total = 3


fe_generator:   0%|          | 0/3 [00:00<?, ?it/s]

[0001] y_shift_1 = roll(shift, ts_code, twap_ret, trade_date, 1)                                 time = 0.03
[0002] y_shift_2 = roll(shift, ts_code, twap_ret, trade_date, 2)                                 time = 0.03
[0003] y_shift_3 = roll(shift, ts_code, twap_ret, trade_date, 3)                                 time = 0.03
[init] <---------- typer_config ----------> 
col_id              7
col_tid            10
col_label           5
cols_tid            1
cols_not            1
cols_label          2
cols_mask           4
cols_num           24
cols_cat            2
cols_datetime       0
cols_timestamp      0
cols_fix            0
name_list_basic     3
cols_fea            0
data_config         6
[+] total = 2


fe_generator:   0%|          | 0/2 [00:00<?, ?it/s]

[0001] x(ts_code)                                                                                time = 0.24
[0002] x(trade_date)                                                                             time = 0.00
[+] fea = 26
[+] history = 0
[+] done!


# Pool

In [6]:
from runningz.res.er.pooler import Pooler

# main

In [7]:
params_res = dict(
    seed = 42,               # random seed
    n_estimators = 1000,     # n rounds
    
    learning_rate = 1.0,     # dy = (1 - lr) * dy + lr * (y - y')
    num_leaves = 200,        # n trys in 1 round
    max_depth = 6,
    
    bagging_freq = 10,       # rounds to bagging sample, sample to choose best try
    bagging_fraction = 0.5,  # bagging ratio
    feature_fraction = 0.5,  # feature sampling in each round

    max_bin = 32,            # split into 16 bins, to calc follow metrics
    lambda_l1 = 0.001,       # bins which ic_i < L1, set as 0
    lambda_l2 = 0.1,         # mean(IC_bins) - L2 * std(IC_bins)
    min_pos_bins = 0.5,      # mean(IC_bins > 0) >= 
    min_gain_round = 0.0001, # IC(y, y_pred_new) - IC(y, y_pred_old) >= 
    
    max_xi_std = 0.05,       # (xi_std - 1).abs() <= max_xi_std
    max_corr = 0.8,          # corr(X_cur, X_add) <= 0.8
    
    early_stopping = -1,     # 10 rounds without better eval, than stop
    extra = False,           # random pick op(args)
)

In [8]:
cols_fea = []
mask_look = 'look'
dm.shape()

{'T0': (5322190, 35), 'T0.ts_code': (5469, 1), 'T0.trade_date': (1214, 1)}

In [9]:
dm.masker.mask_DICT['look'].sum(), dm.masker.mask_DICT['train'].sum()

(2934093, 832603)

In [10]:
cols_num = fm.typer.get_cols_all(dtype = 'num')
fm.typer.typer_config['cols_num'] = cols_num[:24]

cols_num = fm.typer.get_cols_all(dtype = 'num')
cols_cat = fm.typer.get_cols_all(dtype = 'cat')
print(cols_cat)
cols_num

['x(ts_code)', 'x(trade_date)']


['open',
 'high',
 'low',
 'close',
 'pre_close',
 'change',
 'pct_chg',
 'vol',
 'amount',
 'vwap',
 'twap',
 'mean',
 'pre_twap',
 'twap_ret',
 'pre_vwap',
 'vwap_ret',
 'pre_open',
 'open_ret',
 'close_ret',
 'pre_mean',
 'mean_ret',
 'y_shift_1',
 'y_shift_2',
 'y_shift_3']

In [11]:
len(cols_fea), len(fm.typer.get_cols_all(dtype = 'num'))

(0, 24)

In [12]:
rz.get_mem_cur(); rz.get_mem_max()

[2024/02/11 22:51:52] mem_cur = 3.91GB = 4,196,605,952
[2024/02/11 22:51:52] mem_max = 3.95GB = 4,239,724,544


4239724544

In [55]:
i_round = 0
study = Pooler.get_study(study_name = f'round_{i_round}', seed = params_res['seed'] + i_round) # ***seed***
pooler = Pooler(study, cols_num, cols_cat, params_res['max_depth']) # ***max_depth***
path_config = fm_config_list[0]['path_config_list'][0]
fm_config = fm_config_list[0]

[I 2024-02-11 23:04:16,972] A new study created in memory with name: round_0


In [56]:
name_list = []
np.random.seed(42)
for i in tqdm(range(10)):
    score = np.random.random()
    name_dict_list = pooler.run(fm, path_config)
    pooler.study.tell(name_dict_list[0]['trial'], score)
    name_list.append(name_dict_list[0]['name'])
    # print(study.best_value, score)
    print(name_dict_list[0]['trial'])

  0%|          | 0/10 [00:00<?, ?it/s]

In [140]:
import optuna
study = optuna.create_study(
    direction = 'maximize', 
    sampler = optuna.samplers.TPESampler(seed = 42),
    pruner = optuna.pruners.NopPruner(),
)
xs = np.zeros(1000)
for i in tqdm(range(len(xs))):
    trial = study.ask()
    x = 0
    x += trial.suggest_categorical(f'x0', range(100))
    x += trial.suggest_categorical(f'x1', range(100))
    x += trial.suggest_categorical(f'x2', range(100))
    x += trial.suggest_categorical(f'x3', range(100))
    study.tell(trial, x)
    xs[i] = x

[I 2024-02-11 23:41:30,519] A new study created in memory with name: no-name-947a1fde-919e-4c06-bafb-4bb25d729b9a


  0%|          | 0/1000 [00:00<?, ?it/s]

In [142]:
print(study.trials_dataframe())

In [2]:
import optuna
#optuna.samplers.NSGAIISampler()
optuna.samplers.TPESampler()

<ipython-input-2-36da85319a25>:2: FutureWarning: MOTPESampler has been deprecated in v2.9.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.9.0.
  optuna.samplers.MOTPESampler()


In [132]:
# xs2 = xs

In [135]:
xs[50:60]

array([223., 244., 190., 178., 217., 222., 138., 217., 250., 266.])

In [134]:
(xs2 - xs)[:60]

array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    3.,  -55.,   24.,   55.,
        -26.,    2.,   37.,  -20.,    6., -108.])

In [101]:
# xs2

In [120]:
1000 * 25

25000

In [47]:
# name_list_2 = name_list

In [50]:
name_list[-10:]

['__TEMP__ = log1p(pre_twap)',
 '__TEMP__ = amn(close_ret, change)',
 '__TEMP__ = agg(cov, trade_date, open_ret, close_ret)',
 '__TEMP__ = roll(mean_BF_cutpos, ts_code, y_shift_2, trade_date, 60)',
 '__TEMP__ = adt(change, amount)',
 '__TEMP__ = gmean_w_v(change, y_shift_2, pre_close, 0.54)',
 '__TEMP__ = std_m(high,pre_twap,pre_mean,twap,close_ret)',
 '__TEMP__ = invmean_w(pct_chg, vol, y_shift_1)',
 '__TEMP__ = roll(mean, ts_code, pre_close, trade_date, 115)',
 '__TEMP__ = roll(proj_diff, ts_code, close_ret, amount, trade_date, 45)']

In [51]:
name_list_2[-10:]

['__TEMP__ = log1p(pre_twap)',
 '__TEMP__ = amn(close_ret, change)',
 '__TEMP__ = agg(cov, trade_date, open_ret, close_ret)',
 '__TEMP__ = roll(mean_BF_cutpos, ts_code, y_shift_2, trade_date, 60)',
 '__TEMP__ = adt(change, amount)',
 '__TEMP__ = gmean_w_v(change, y_shift_2, pre_close, 0.54)',
 '__TEMP__ = std_m(high,pre_twap,pre_mean,twap,close_ret)',
 '__TEMP__ = invmean_w(pct_chg, vol, y_shift_1)',
 '__TEMP__ = roll(mean, ts_code, pre_close, trade_date, 115)',
 '__TEMP__ = roll(proj_diff, ts_code, close_ret, amount, trade_date, 45)']

In [15]:
print(rz.dict_2_str(params_res))
df_history = []
cols_fea = []
X_cur = None
ic_cur = 0
ic_eval = 0
Model = rz.model.LRModel

for i_round in range(params_res['n_estimators']): # ***n_estimators***
    # if i_round < 206: continue
    
    # -------------------------------------------------- ***bagging_fraction***
    if 0 < params_res['bagging_fraction'] < 1:
        mask_sample = 'bagging_sample'
        if i_round % params_res['bagging_freq'] == 0:
            mask = dm.masker.mask_DICT[mask_look].copy()
            mask &= np.random.random(len(mask)) < params_res['bagging_fraction']
            dm.set_mask(mask, mask_sample)            
    else:
        mask_sample = mask_look

    # -------------------------------------------------- ***bagging_freq***
    if (i_round == 0) or (i_round % params_res['bagging_freq'] == 0): 
        y = dm.get_col_numpy(col_label, mask_name = mask_sample)
        if len(cols_fea) == 0:
            X_cur = None
            dy = y
        else:
            X_cur = dm.get_cols_numpy_cszscore([col_tid], cols_fea, mask_name = mask_sample)
            model = Model(dtype = np.float32).train(dict(X_train = X_cur, y_train = y))
            y_pred = model.predict(X_cur)
            dy = y - y_pred 
    
    # fix sample
    dm_i = fm.dm.get_cols_pandas([col_tid, col_label], mask_name = mask_sample)
    dm_i = rz.DataMaster(dm_i)
    dm_i.set_by_col('__LABEL__', dy)
    
    # -------------------------------------------------- ***feature_fraction***
    cols_num = fm.typer.get_cols_all(dtype = 'num')
    cols_cat = fm.typer.get_cols_all(dtype = 'cat')
    if 0 < params_res['feature_fraction'] < 1:
        cols_num = list(pd.Series(cols_num).sample(frac = params_res['feature_fraction']))
        cols_cat = list(pd.Series(cols_cat).sample(frac = params_res['feature_fraction']))
    
    # ------------------------------------------------------------------------------------------ pooler
    df_name_score = []

    study = Pooler.get_study(study_name = f'round_{i_round}', seed = params_res['seed'] + i_round) # ***seed***
    pooler = Pooler(study, cols_num, cols_cat, params_res['max_depth']) # ***max_depth***
    path_config = fm_config_list[0]['path_config_list'][0]
    fm_config = fm_config_list[0]
    fm_config['fm_config_i'] = 999
    
    for i_try in tqdm(range(params_res['num_leaves'])):
        # xi
        name_dict_list = pooler.run(fm, path_config, fm_config = fm_config)   
        xi = dm.get_cols_numpy_cszscore([col_tid], ['__TEMP__'], mask_name = mask_sample)
        dm_i.set_by_col('__PRED__', xi.reshape(-1))
        fm.dm.drop([name_dict_list[0]['name']], verbose = False)
        
        # df_ic_i
        df_ic_i = get_df_ic(dm_i, col_tid, params_res['max_bin'])
        sign_ic, ic_raw = np.sign(df_ic_i.mean()), df_ic_i.mean()

        # ***lambda_l1***
        s_ic = sign_ic * df_ic_i
        s_ic[(s_ic >= 0) & (s_ic < params_res['lambda_l1'])] = 0
        ic = s_ic.mean()

        # ***lambda_l2***
        ic_reg = params_res['lambda_l2'] * s_ic.std(ddof = 0)
        score_l2 = ic - ic_reg

        # ***min_pos_bins***
        pos_bins = (s_ic > 0).mean()

        # ***std***, *************
        xi_std = rz.util.npy_numba.np_std(xi.reshape(-1))
        
        # ***max_corr***
        max_corr = 0
        if X_cur is not None:
            max_corr = np.max(np.abs(numba_corr_x_a(xi.reshape(-1), X_cur)))
        score = score_l2 * (2 / (1 + max_corr)) ** 0.5
        
        # ***extra***
        if not params_res['extra']: 
            pooler.study.tell(name_dict_list[0]['trial'], score)  
            
        name = name_dict_list[0]['name'][len('__TEMP__ = '):]
        df_name_score.append([score, score_l2, ic, ic_reg, pos_bins, ic_raw, xi_std, max_corr, name])

    cols = ['score', 'score_l2', 'ic', 'ic_reg', 'pos_bins', 'ic_raw', 'xi_std', 'max_corr', 'name']
    df_name_score = pd.DataFrame(df_name_score, columns = cols)
    
    # ------------------------------ filter    
    shape_origin = df_name_score.shape
    df_name_score = df_name_score[(df_name_score['xi_std'] - 1).abs() <= params_res['max_xi_std']] # ***max_xi_std***
    shape_xi_std = df_name_score.shape
    df_name_score = df_name_score[(df_name_score['max_corr'] <= params_res['max_corr'])]           # ***max_corr***
    shape_max_corr = df_name_score.shape
    df_name_score = df_name_score[df_name_score['pos_bins'] >= params_res['min_pos_bins']]         # ***min_pos_bins***
    shape_min_pos_bins = df_name_score.shape
    print(f'o={shape_origin}, std={shape_xi_std}, corr={shape_max_corr}, pos={shape_min_pos_bins}')
    if len(df_name_score) == 0: continue
    df_name_score = df_name_score.sort_values('score', ascending = False)
    cols = ['score', 'score_l2', 'ic', 'ic_reg', 'pos_bins', 'ic_raw', 'xi_std', 'max_corr']
    print(df_name_score.head(3)[cols].to_string())
    
    # ------------------------------------------------------------------------------------------ try top
    find = False
    for i_try in range(min(10, len(df_name_score))):
        # X_try
        name = df_name_score.iloc[i_try]['name']
        dm.fe_go([name], verbose = False, again = True)
        X_add = dm.get_cols_numpy_cszscore([col_tid], [name], mask_name = mask_sample)
        X_try = X_add
        if X_cur is not None: 
            X_try = rz.util.npy_numba.np_hstack_F(X_cur, X_add)
        if i_try == 0:
            print(f'[+]', X_cur.shape if X_cur is not None else None, \
                  X_try.shape, y.shape, dy.shape, df_name_score.shape)
        print(f'[+] mean_std, {np.mean(X_add):.6f}, {np.std(X_add):.6f}')
        
        # debug
        if df_name_score.iloc[i_try]['score'] > 0.25: raise RuntimeError("score > 0.25")
        
        # model_try
        model_try = Model(dtype = np.float32).train(dict(X_train = X_try, y_train = y))
        y_pred_try = model_try.predict(X_try)
        dy_try = y - y_pred_try    
        s_corr_res = pd.Series([rz.metric.corr_fast(dy_try, X_try[:, i]) for i in range(X_try.shape[1])])
        print(f'[+] corr with res, min = {s_corr_res.min():.6f}, max = {s_corr_res.max():.6f}')
        
        # ic_try
        dm_i.set_by_col('__LABEL__', y)
        dm_i.set_by_col('__PRED__', y_pred_try)
        df_ic_i = get_df_ic(dm_i, col_tid, params_res['max_bin'])
        ic_try = np.abs(df_ic_i.mean())
        
        # ***min_gain_round***
        if ic_try - ic_cur >= params_res['min_gain_round']:
            cols_fea += [name]
            fm.typer.add(name, dtype = 'num')
            print(rz.util.mem.s_green(f'[+] IC = {ic_cur:.6f} -> {ic_try:.6f}, +++, cols_fea = {len(cols_fea)}'))
            
             # ***learning_rate***
            dy = (1 - params_res['learning_rate']) * dy + params_res['learning_rate'] * dy_try
            # dy = dy_try
            X_cur = X_try
            ic_cur = ic_try
            ic_eval, df_ic_i_eval = get_score_eval(fm, model_try, cols_fea, params_res['max_bin'], mask_name = 'eval')
            print(rz.util.mem.s_green(f'[+] ic_eval = {ic_eval:.6f}, i_try = {i_try}')); sys.stdout.flush()
            print(f'[+] name = {name}')
            find = True
            break # find one
        else:
            dm.drop([name], verbose = False)
            print(rz.util.mem.s_red(f'[+] IC = {ic_cur:.6f} -> {ic_try:.6f}, ---, cols_fea = {len(cols_fea)}'))
    
    # ***bagging_freq***
    if not find and (i_round % params_res['bagging_freq'] == 0):
        ic_cur = ic_try
        print(rz.util.mem.s_green(f'[+] IC = {ic_cur:.6f} -> {ic_try:.6f}, ===, cols_fea = {len(cols_fea)}'))
    
    # ***early_stopping***
    df_history.append([i_round, ic_cur, ic_eval, find, name if find else None])
    a_ic_eval = [x[2] for x in df_history]
    es = params_res.get('early_stopping', None)

    i_round_base = i_round // params_res['bagging_freq'] * params_res['bagging_freq']
    if i_round_base == len(a_ic_eval): argmax = i_round_base - 1
    else:  argmax = i_round_base + np.argmax(list(a_ic_eval[i_round_base:]))
    print(f'[+] i - argmax = {i_round} - {argmax} = {i_round - argmax}, es = {es}')
    if es is not None and es > 0 and i_round - argmax >= es:
        print('[+] early_stopping!'); sys.stdout.flush()
        break
    rz.get_mem_cur(); rz.get_mem_max(); sys.stdout.flush()

df_history = pd.DataFrame(df_history, columns = ['i', 'ic_cur', 'ic_eval', 'find', 'name'])

[I 2024-02-11 10:52:45,577] A new study created in memory with name: round_0


{
    "seed": 42,
    "n_estimators": 1000,
    "learning_rate": 1.0,
    "num_leaves": 200,
    "max_depth": 6,
    "bagging_freq": 10,
    "bagging_fraction": 0.5,
    "feature_fraction": 0.5,
    "max_bin": 32,
    "lambda_l1": 0.001,
    "lambda_l2": 0.1,
    "min_pos_bins": 0.5,
    "min_gain_round": 0.0001,
    "max_xi_std": 0.05,
    "max_corr": 0.8,
    "early_stopping": -1,
    "extra": false
}


  0%|          | 0/200 [00:00<?, ?it/s]

o=(200, 9), std=(139, 9), corr=(139, 9), pos=(129, 9)
       score  score_l2       ic   ic_reg  pos_bins    ic_raw   xi_std  max_corr
81  0.030709  0.021714 0.027556 0.005842  0.593750  0.027556 1.000000         0
121 0.027892  0.019723 0.025880 0.006157  0.562500 -0.025880 1.000000         0
133 0.025232  0.017842 0.020636 0.002795  0.781250 -0.020636 1.000000         0
[+] None (1466746, 1) (1466746,) (1466746,) (129, 9)
[+] mean_std, 0.000000, 1.000000
[+] corr with res, min = -0.000000, max = -0.000000
[+] IC = 0.000000 -> 0.027556, +++, cols_fea = 1
[+] ic_eval = 0.030950, i_try = 0
[+] name = roll(diff_sum, ts_code, close_ret, pct_chg, trade_date, 40)
[+] i - argmax = 0 - 0 = 0, es = -1
[2024/02/11 10:54:38] mem_cur = 4.36GB = 4,678,627,328
[2024/02/11 10:54:38] mem_max = 4.36GB = 4,678,627,328


[I 2024-02-11 10:54:38,149] A new study created in memory with name: round_1


...

  0%|          | 0/200 [00:00<?, ?it/s]

o=(200, 9), std=(147, 9), corr=(104, 9), pos=(65, 9)
       score  score_l2       ic   ic_reg  pos_bins    ic_raw   xi_std  max_corr
79  0.004703  0.003537 0.004874 0.001337  0.656250 -0.004907 1.000000  0.131101
23  0.004028  0.002966 0.005279 0.002313  0.531250 -0.005290 1.000000  0.084565
110 0.002943  0.002614 0.003554 0.000940  0.593750  0.003596 1.000000  0.577743
[+] (1466063, 79) (1466063, 80) (1466063,) (1466063,) (65, 9)
[+] mean_std, 0.000000, 1.000000
[+] corr with res, min = -0.000000, max = 0.000000
[+] IC = 0.072211 -> 0.072368, +++, cols_fea = 80
[+] ic_eval = 0.052934, i_try = 0
[+] name = amean_w(y_shift_2, gmean_w_v(roll(skew__ndx__max, ts_code, pct_chg, trade_date, 45), roll(beta, ts_code, y_shift_1, change, trade_date, 45), mean_ret, 0.64), roll(skew_BF_power, ts_code, close, trade_date, 70))
[+] i - argmax = 82 - 82 = 0, es = -1
[2024/02/11 11:56:30] mem_cur = 12.52GB = 13,446,340,608
[2024/02/11 11:56:30] mem_max = 13.08GB = 14,044,721,152


[I 2024-02-11 11:56:30,516] A new study created in memory with name: round_83


  0%|          | 0/200 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
# df_history = pd.DataFrame(df_history, columns = ['i', 'ic_cur', 'ic_eval', 'find', 'name'])

# result

In [ ]:
print(rz.util.mem.s_green(f'[+] IC = {ic_cur:.6f} -> {ic_try:.6f}, +++, cols_fea = {len(cols_fea)}'))
print(rz.util.mem.s_green(f'[+] ic_eval = {ic_eval:.6f}, i_try = {i_try}')); sys.stdout.flush()

In [ ]:
print(rz.util.mem.s_green(f'[+] IC = {ic_cur:.6f} -> {ic_try:.6f}, +++, cols_fea = {len(cols_fea)}'))
print(rz.util.mem.s_green(f'[+] ic_eval = {ic_eval:.6f}, i_try = {i_try}')); sys.stdout.flush()

In [ ]:
# Model = rz.model.LRModel
# model_try = Model(dtype = np.float64).train(dict(X_train = X_try, y_train = y))
# y_pred_try = model_try.predict(X_try)
# dy_try = y - y_pred_try  

In [ ]:
# print(pd.Series([rz.metric.corr_fast(dy_try, X_try[:, i]) for i in range(X_try.shape[1])]).to_string())

In [34]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as F1, precision_score as P, recall_score as R

# 关于波动的简要分析

结论: 样本量过小的情况下, 在测试集合, 多预测准一个, 对结果影响很大

In [104]:
n = 240            # 测试集样本数
n1 = int(n * 0.14) # 正样本数量
m1 = int(n * 0.16) # 预测为正样本的数量, 权衡 P & R
print(n1, m1)

33 38


In [105]:
# 预测中 25个正样本, 最终得分 = 70.4
a = [1] * n1 + [0] * (n - n1)
b = [1] * 25 + [0] * (n - m1) + [1] * (m1 - 25)
print(f'P = {P(a, b):.4f}, R = {R(a, b):.4f}, F1 = {F1(a, b):.4f}')

P = 0.6579, R = 0.7576, F1 = 0.7042


In [106]:
# 预测中 26个正样本, 最终得分 = 73.2
a = [1] * n1 + [0] * (n - n1)
b = [1] * 26 + [0] * (n - m1) + [1] * (m1 - 26)
print(f'P = {P(a, b):.4f}, R = {R(a, b):.4f}, F1 = {F1(a, b):.4f}')

P = 0.6842, R = 0.7879, F1 = 0.7324


In [1]:
len('----------------------------------------------------------------------------------------')

88